# Create Transformed Test Data
A notebook for creating OpenRefine procedures that change tabular COUNTER reports into dataframes for use in testing.

In [67]:
from pathlib import Path
from itertools import chain


new_field_names = {
    "Metric_Type": "metric_type",
    "Metric Type": "metric_type",
    "User_Activity": "metric_type",
    "User Activity": "metric_type",
    "Access Denied Category": "metric_type",
    "Access denied category": "metric_type",
    "Column": "resource_name",
    "Book DOI": "DOI",
    "Journal": "resource_name",
    "Journal DOI": "DOI",
    "Collection": "resource_name",
    "Content Provider": "publisher",
    "Data_Type": "data_type",
    "Access_Method": "access_method",
    "Database": "resource_name",
    "Publisher_ID": "publisher_ID",
    "Title": "resource_name",
    "Print_ISSN": "print_ISSN",
    "Online_ISSN": "online_ISSN",
    "Print ISSN": "print_ISSN",
    "Online ISSN": "online_ISSN",
    "Section_Type": "section_type",
    "Access_Type": "access_type",
    "Item": "resource_name",
    "Authors": "authors",
    "Publication_Date": "publication_date",
    "Article_Version": "article_version",
    "Parent_Title": "parent_title",
    "Parent_Authors": "parent_authors",
    "Parent_Publication_Date": "parent_publication_date",
    "Parent_Article_Version": "parent_article_version",
    "Parent_Data_Type": "parent_data_type",
    "Parent_DOI": "parent_DOI",
    "Parent_Proprietary_ID": "parent_proprietary_ID",
    "Parent_ISBN": "parent_ISBN",
    "Parent_Print_ISSN": "parent_print_ISSN",
    "Parent_Online_ISSN": "parent_online_ISSN",
    "Parent_URI": "parent_URI",
    "Platform": "platform",
    "Publisher": "publisher",
    "Proprietary_ID": "proprietary_ID",
    "Proprietary Identifier": "proprietary_ID",
    "ISSN": "online_ISSN"
}

fields_added_by_report = {
    'PR1': ['data_type'],
    'DB1': ['data_type'],
    'DB2': ['data_type'],
    'BR1': ['data_type', 'section_type', 'metric_type'],
    'BR2': ['data_type', 'section_type', 'metric_type'],
    'JR1': ['data_type', 'section_type', 'metric_type'],
    'BR3': ['data_type', 'section_type'],
    'JR2': ['data_type', 'section_type'],
    'MR1': ['data_type', 'metric_type'],
    'PR': [],
    'DR': [],
    'TR': [],
    'IR': [],
}

def split_pasted_field_list(pasted_field_list):
    field_list = []
    for field in pasted_field_list.split('\t'):
        if field == "" or field == " ":
            field_list.append("Column")
        else:
            field_list.append(field)
    return field_list

In [68]:
def add_stats_source_and_report_type_fields(statistics_source_ID, report_type):
    return f"""
        {{
        "op": "core/column-addition",
        "engineConfig": {{
            "facets": [],
            "mode": "row-based"
        }},
        "baseColumnName": "Platform",
        "expression": "grel:'{report_type}'",
        "onError": "set-to-blank",
        "newColumnName": "report_type",
        "columnInsertIndex": 0,
        "description": "Create column ``report_type`` with the report type"
    }},
    {{
        "op": "core/column-addition",
        "engineConfig": {{
            "facets": [],
            "mode": "row-based"
        }},
        "baseColumnName": "Platform",
        "expression": "grel:{statistics_source_ID}",
        "onError": "set-to-blank",
        "newColumnName": "statistics_source_ID",
        "columnInsertIndex": 0,
        "description": "Create column ``statistics_source_ID`` with statistics source ID"
    }},
    """


def pivot_dates_and_usage_data():
    return """
        {
            "op": "core/transpose-columns-into-rows",
            "startColumnName": "Reporting_Period_Total",
            "columnCount": -1,
            "ignoreBlankCells": true,
            "fillDown": true,
            "separator": null,
            "keyColumnName": "usage_date",
            "valueColumnName": "usage_count",
            "description": "Pivot all columns with counts into two columns, one named ``usage_date`` with the column name/date and the other named ``usage_count`` with the data value/count"
        },
        {
            "op": "core/transpose-columns-into-rows",
            "startColumnName": "Reporting Period Total",
            "columnCount": -1,
            "ignoreBlankCells": true,
            "fillDown": true,
            "separator": null,
            "keyColumnName": "usage_date",
            "valueColumnName": "usage_count",
            "description": "Pivot all columns with counts into two columns, one named ``usage_date`` with the column name/date and the other named ``usage_count`` with the data value/count"
        },
        {
            "op": "core/row-removal",
            "engineConfig": {
                "facets": [
                    {
                        "type": "list",
                        "name": "usage_date",
                        "expression": "value",
                        "columnName": "usage_date",
                        "invert": false,
                        "omitBlank": false,
                        "omitError": false,
                        "selection": [
                            {
                                "v": {
                                    "v": "Reporting_Period_Total",
                                    "l": "Reporting_Period_Total"
                                }
                            },
                            {
                                "v": {
                                    "v": "Reporting_Period_HTML",
                                    "l": "Reporting_Period_HTML"
                                }
                            },
                            {
                                "v": {
                                    "v": "Reporting_Period_PDF",
                                    "l": "Reporting_Period_PDF"
                                }
                            },
                            {
                                "v": {
                                    "v": "Reporting Period Total",
                                    "l": "Reporting Period Total"
                                }
                            },
                            {
                                "v": {
                                    "v": "Reporting Period HTML",
                                    "l": "Reporting Period HTML"
                                }
                            },
                            {
                                "v": {
                                    "v": "Reporting Period PDF",
                                    "l": "Reporting Period PDF"
                                }
                            }
                        ],
                        "selectBlank": false,
                        "selectError": false
                    }
                ],
                "mode": "row-based"
            },
            "description": "Remove total rows based on total metrics"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "usage_count",
            "expression": "value.toNumber()",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Make cells in column ``usage_count`` numbers"
        },
        {
            "op": "core/row-removal",
            "engineConfig": {
                "facets": [
                    {
                        "type": "list",
                        "name": "usage_count",
                        "expression": "grel:or(isBlank(value),value==0)",
                        "columnName": "usage_count",
                        "invert": false,
                        "omitBlank": false,
                        "omitError": false,
                        "selection": [
                            {
                                "v": {
                                    "v": true,
                                    "l": "true"
                                }
                            }
                        ],
                        "selectBlank": false,
                        "selectError": false
                    }
                ],
                "mode": "row-based"
            },
            "description": "Remove rows with no usage"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "usage_date",
    """ + r"""
            "expression": "grel:if(type(value)=='date',value,if(isError(value.toDate()),if(isNull(value.match(/(\\w{3})\\-(\\d{2})?(\\d{2})/)[1]),'20'+value.match(/(\\w{3})\\-(\\d{2})/)[1],value.match(/\\w{3}\\-(\\d{4})/)[0])+'-'+value.match(/(\\w{3})\\-(\\d{2})?(\\d{2})/)[0].replace('Jan','01').replace('Feb','02').replace('Mar','03').replace('Apr','04').replace('May','05').replace('Jun','06').replace('Jul','07').replace('Aug','08').replace('Sep','09').replace('Oct','10').replace('Nov','11').replace('Dec','12')+'-01',value))",
    """ + """
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure dates in column ``usage_date`` can be changed into date data types"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "usage_date",
            "expression": "grel:value.split('T')[0].toDate()",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change the values in column ``usage_date`` to date data type"
        },
    """

In [69]:
def add_PR1_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Platform'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:null",
            "onError": "set-to-blank",
            "newColumnName": "resource_name",
            "columnInsertIndex": 1,
            "description": "Create column ``resource_name`` with null values"
        },
    """


def add_DB1_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Database'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
    """


def add_DB2_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Database'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('licence','license')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have `license` always use American English spelling"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('denied. C', 'denied: c').replace('denied.', 'denied:').replace('denied: C', 'denied: c')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have turnaway report metrics always use a colon and a lowercase `c`"
        },
    """


def add_BR1_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book'",
            "onError": "set-to-blank",
            "newColumnName": "section_type",
            "columnInsertIndex": 1,
            "description": "Create column ``section_type`` with the section type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Successful Title Requests'",
            "onError": "set-to-blank",
            "newColumnName": "metric_type",
            "columnInsertIndex": 1,
            "description": "Create column ``metric_type`` with the metric type"
        },
    """


def add_BR2_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book_Segment'",
            "onError": "set-to-blank",
            "newColumnName": "section_type",
            "columnInsertIndex": 1,
            "description": "Create column ``section_type`` with the section type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Successful Section Requests'",
            "onError": "set-to-blank",
            "newColumnName": "metric_type",
            "columnInsertIndex": 1,
            "description": "Create column ``metric_type`` with the metric type"
        },
    """


def add_BR3_fields():  #Also BR5
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Book'",
            "onError": "set-to-blank",
            "newColumnName": "section_type",
            "columnInsertIndex": 1,
            "description": "Create column ``section_type`` with the section type"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('licence','license')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have `license` always use American English spelling"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('denied. C', 'denied: c').replace('denied.', 'denied:').replace('denied: C', 'denied: c')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have turnaway report metrics always use a colon and a lowercase `c`"
        },
    """


def add_JR1_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Journal'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Article'",
            "onError": "set-to-blank",
            "newColumnName": "section_type",
            "columnInsertIndex": 1,
            "description": "Create column ``section_type`` with the section type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Successful Full-text Article Requests'",
            "onError": "set-to-blank",
            "newColumnName": "metric_type",
            "columnInsertIndex": 1,
            "description": "Create column ``metric_type`` with the metric type"
        },
    """


def add_JR2_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Journal'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Article'",
            "onError": "set-to-blank",
            "newColumnName": "section_type",
            "columnInsertIndex": 1,
            "description": "Create column ``section_type`` with the section type"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('licence','license')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have `license` always use American English spelling"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "metric_type",
            "expression": "grel:value.replace('denied. C', 'denied: c').replace('denied.', 'denied:').replace('denied: C', 'denied: c')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Have turnaway report metrics always use a colon and a lowercase `c`"
        },
    """


def add_MR1_fields():
    return """
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Multimedia'",
            "onError": "set-to-blank",
            "newColumnName": "data_type",
            "columnInsertIndex": 1,
            "description": "Create column ``data_type`` with the data type"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'Successful Content Unit Requests'",
            "onError": "set-to-blank",
            "newColumnName": "metric_type",
            "columnInsertIndex": 1,
            "description": "Create column ``metric_type`` with the metric type"
        },
    """

In [70]:
def adjust_item_report_fields():
    return """
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "publication_date",
            "expression": "grel:value.split('T')[0].toDate()",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change the values in column ``publication_date`` to date data type without time zones"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "parent_publication_date",
            "expression": "grel:value.split('T')[0].toDate()",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change the values in column ``parent_publication_date`` to date data type without time zones"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "publication_date",
            "expression": "grel:if(or(value.toString('yyyy-MM-DD')=='1000-01-01',value.toString('yyyy-MM-DD')=='1753-01-01',value.toString('yyyy-MM-DD')=='1900-01-01'),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Null values in column ``publication_date`` if the dates are the RDBMS/spreadsheet minimum date data type values `1000-01-01`, `1753-01-01`, or `1900-01-01`"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "parent_publication_date",
            "expression": "grel:if(or(value.toString('yyyy-MM-DD')=='1000-01-01',value.toString('yyyy-MM-DD')=='1753-01-01',value.toString('yyyy-MM-DD')=='1900-01-01'),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Null values in column ``parent_publication_date`` if the dates are the RDBMS/spreadsheet minimum date data type values `1000-01-01`, `1753-01-01`, or `1900-01-01`"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "authors",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``authors`` are encoded with UTF-8"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "parent_title",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``parent_title`` are encoded with UTF-8"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "parent_authors",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``parent_authors`` are encoded with UTF-8"
        },
    """


def make_final_field_adjustments():
    return """
        
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "DOI",
            "expression": "grel:if(or(value=='',value==' '),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change whitespace values to null in column ``DOI``"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "proprietary_ID",
            "expression": "grel:if(or(value=='',value==' '),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change whitespace values to null in column ``proprietary_ID``"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "online_ISSN",
            "expression": "grel:if(or(value=='',value==' '),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change whitespace values to null in column ``online_ISSN``"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "print_SSN",
            "expression": "grel:if(or(value=='',value==' '),null,value)",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Change whitespace values to null in column ``print_ISSN``"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "resource_name",
            "expression": "grel:value.replace(/\\n/,'')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Remove errant newlines from ``resource_name`` values"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "resource_name",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``resource_name`` are encoded with UTF-8"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "publisher",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``publisher`` are encoded with UTF-8"
        },
        {
            "op": "core/text-transform",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "columnName": "platform",
            "expression": "grel:value.reinterpret('utf-8').unescape('html')",
            "onError": "keep-original",
            "repeat": false,
            "repeatCount": 10,
            "description": "Ensure values in column ``platform`` are encoded with UTF-8"
        },
        {
            "op": "core/column-reorder",
            "columnNames": [
                "statistics_source_ID",
                "report_type",
                "resource_name",
                "publisher",
                "publisher_ID",
                "platform",
                "authors",
                "publication_date",
                "article_version",
                "DOI",
                "proprietary_ID",
                "ISBN",
                "print_ISSN",
                "online_ISSN",
                "URI",
                "data_type",
                "section_type",
                "YOP",
                "access_type",
                "access_method",
                "parent_title",
                "parent_authors",
                "parent_publication_date",
                "parent_article_version",
                "parent_data_type",
                "parent_DOI",
                "parent_proprietary_ID",
                "parent_ISBN",
                "parent_print_ISSN",
                "parent_online_ISSN",
                "parent_URI",
                "metric_type",
                "usage_date",
                "usage_count"
            ],
            "description": "Reorder columns"
        },
        {
            "op": "core/column-addition",
            "engineConfig": {
                "facets": [],
                "mode": "row-based"
            },
            "baseColumnName": "statistics_source_ID",
            "expression": "grel:'['+forEach(row.columnNames,field,if(isBlank(cells[field].value),'None',if(cells[field].value.type()=='date','\\\"'+cells[field].value.toString('yyyy-MM-dd')+'\\\"',if(cells[field].value.type()=='string','\\\"'+cells[field].value.replace('\\\"','\\\\\\\"')+'\\\"',cells[field].value.round())))).join(', ')+'],'",
            "onError": "set-to-blank",
            "newColumnName": "df",
            "columnInsertIndex": 0,
            "description": "Create column ``df`` by combining all the other fields and formatting them for insertion into a dataframe"
        },
    """

## 1. Add Workbook `statistics_source_ID`, `report_type` Values and Fields in Reports
For `report_types_and_fields`, create a dict where the keys are the report types and the values are strings of the fields from those reports up to the usage data (either the reporting totals or the first month) copied from the given sheets.

In [71]:
statistics_source_ID = 0
report_types_and_fields = {
    #
}

## 2. Generate List of `report_type` and JSONs for Each Worksheet

In [72]:
report_types = [report for report in report_types_and_fields.keys()]
project_JSONs = {}
for report in report_types:
    project_JSONs[report] = Path(f'transform_test_data_{report}.json')

for report_type, JSON in project_JSONs.items():
    with open(JSON, 'w') as file:
        file.write(add_stats_source_and_report_type_fields(statistics_source_ID, report_type))
        file.write(pivot_dates_and_usage_data())

## 3. Generate List of Fields for Dataframe

In [73]:
all_fields_list = []
for field_list in report_types_and_fields.values():
    all_fields_list.append(split_pasted_field_list(field_list))
all_fields_list = list(chain.from_iterable(all_fields_list))
field_set = set()
for field in all_fields_list:
    if new_field_names.get(field):
        field_set.add(new_field_names[field])
    else:
        field_set.add(field)
for report in report_types:
    for field in fields_added_by_report[report]:
        field_set.add(field)
renamed_all_fields_list = list(field_set)

## 4. Rename Existing Fields and Create Null Fields

In [74]:
for report in report_types:
    new_fields_list = []
    JSON_sections_to_write = []
    original_field_list = split_pasted_field_list(report_types_and_fields[report])
    for field in original_field_list:
        if new_field_names.get(field):
            new_fields_list.append(new_field_names[field])
            JSON_sections_to_write.append(f"""
                {{
                    "op": "core/column-rename",
                    "oldColumnName": "{field}",
                    "newColumnName": "{new_field_names[field]}"
                }},
            """)
        else:
            new_fields_list.append(field)
    if "resource_name" in new_fields_list:
        JSON_sections_to_write.append("""
            {
                "op": "core/row-removal",
                "engineConfig": {
                    "facets": [
                        {
                            "type": "list",
                            "name": "resource_name",
                            "expression": "value",
                            "columnName": "resource_name",
                            "invert": false,
                            "omitBlank": false,
                            "omitError": false,
                            "selection": [
                                {
                                    "v": {
                                        "v": "Total for all journals",
                                        "l": "Total for all journals"
                                    },
                                    "v": {
                                        "v": "Total for all collections",
                                        "l": "Total for all collections"
                                    },
                                    "v": {
                                        "v": "Total for all Titles",
                                        "l": "Total for all Titles"
                                    },
                                    "v": {
                                        "v": "Total for all titles",
                                        "l": "Total for all titles"
                                    }
                                }
                            ],
                            "selectBlank": false,
                            "selectError": false
                        }
                    ],
                    "mode": "row-based"
                },
                "description": "Remove total rows"
            },
        """)
    missing_fields = [f for f in renamed_all_fields_list if f not in new_fields_list]
    missing_fields = [f for f in missing_fields if f not in fields_added_by_report[report]]
    for field in missing_fields:
        JSON_sections_to_write.append(f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "statistics_source_ID",
                "expression": "grel:null",
                "onError": "set-to-blank",
                "newColumnName": "{field}",
                "columnInsertIndex": 2,
                "description": "Create column ``{field}`` with null values"
            }},
        """)
    with open(project_JSONs[report], 'a') as file:
        for string in JSON_sections_to_write:
            file.write(string)

## 5. Add R5 Fields to R4 Reports

In [75]:
for report in report_types:
    if report == 'PR1':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_PR1_fields())
    elif report == 'DB1':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_DB1_fields())
    elif report == 'DB2':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_DB2_fields())
    elif report == 'BR1':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_BR1_fields())
    elif report == 'BR2':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_BR2_fields())
    elif report == 'BR3':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_BR3_fields())
    elif report == 'JR1':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_JR1_fields())
    elif report == 'JR2':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_JR2_fields())
    elif report == 'MR1':
        with open(project_JSONs[report], 'a') as file:
            file.write(add_MR1_fields())

## 6. Adjust R5 Fields

In [76]:
for report in report_types:
    if report == 'TR' or report == 'IR':
        with open(project_JSONs[report], 'a') as file:
            file.write("""
                {
                    "op": "core/text-transform",
                    "engineConfig": {
                        "facets": [],
                        "mode": "row-based"
                    },
                    "columnName": "YOP",
                    "expression": "value.toNumber()",
                    "onError": "keep-original",
                    "repeat": false,
                    "repeatCount": 10,
                    "description": "Change the values in column ``YOP`` to numbers"
                }
            """)
    if report == 'IR':
        with open(project_JSONs[report], 'a') as file:
            file.write(adjust_item_report_fields())

## 7. Finish JSONs

In [77]:
for report in report_types:
        with open(project_JSONs[report], 'a') as file:
            file.write(make_final_field_adjustments())